### Creating SQL Tables for Project Data

In [1]:
#load library
import pandas as pd
import sqlite3 as sq

#connect to the database, which - in the case of sqlite
conn = sq.connect("project_data.db")
c = conn.cursor()

In [2]:
#load dataframe and check data types before creating SQL table
ca_fires = pd.read_csv("california_monthly_county_wildfire_acreage.csv")
ca_fires.dtypes

county             object
year              float64
month             float64
wildfire_acres    float64
dtype: object

In [3]:
#loading in the rest of the data
wa_fires = pd.read_csv("washington_monthly_county_wildfire_acreage.csv")
or_fires = pd.read_csv("oregon_monthly_county_wildfire_acreage.csv")

In [4]:
#renaming a column to match the sql table
or_fires.rename(columns={'EstTotalAcres': 'wildfire_acres'}, inplace=True)

**Creating and filling table for wildfire data**

In [5]:
#create table for county-month level fire data

#drop table first if edits are needed
c.execute(""" DROP TABLE IF EXISTS wildfires """)

#create new wildfires table
c.execute(""" CREATE TABLE wildfires (
            county VARCHAR(50),
            year FLOAT (6),
            month FLOAT(3),
            wildfire_acres FLOAT (25)
            ) """)

In [6]:
#filling the table with data
ca_fires.to_sql('wildfires', conn, if_exists='append', index=False)
wa_fires.to_sql('wildfires', conn, if_exists='append', index=False)
or_fires.to_sql('wildfires', conn, if_exists='append', index=False)

4085

In [7]:
#checking to make sure the data looks good
fires_check = pd.read_sql('SELECT * FROM wildfires', conn)
fires_check

,county,year,month,wildfire_acres
0,Alameda,2000.0,NaN,75.462360
1,Alameda,2001.0,8.0,117.644700
2,Alameda,2002.0,6.0,304.263268
3,Alameda,2003.0,7.0,3.683039
4,Alameda,2003.0,8.0,4056.569846
...,...,...,...,...
7567,Yamhill,2021.0,6.0,0.250000
7568,Yamhill,2021.0,7.0,9.950000
7569,Yamhill,2021.0,8.0,0.020000
7570,Yamhill,2022.0,7.0,1.000000


In [8]:
#inspecting data frame for SQL table
prism = pd.read_csv("all_county_prism_data.csv")
prism.head()
#prism.dtypes
#print(prism.columns)

,county_name,county_fips,year,month,date,precipitation_total_km3,max_temperature_c
0,Wahkiakum,69,2000,1,2000-01-01,0.396535,8.536
1,Wahkiakum,69,2000,2,2000-02-01,0.196691,10.657
2,Wahkiakum,69,2000,3,2000-03-01,0.213708,11.328
3,Wahkiakum,69,2000,4,2000-04-01,0.132705,16.417
4,Wahkiakum,69,2000,5,2000-05-01,0.139677,17.844


**Creating table for prism data**

In [9]:
#drop table if edits are needed
c.execute(""" DROP TABLE IF EXISTS prism """)


#create new table for prism data
c.execute(""" CREATE TABLE prism (
            county_name VARCHAR(50),
            county_fips INT(10),
            year INT(5),
            month INT(3),
            date DATE,
            precipitation_total_km3 FLOAT (25),
            max_temperature_c FLOAT (10)
            ) """)

In [10]:
#filling the table with data
prism.to_sql('prism', conn, if_exists='append', index=False)

33516

In [11]:
#checking to make sure the data looks good
prism_check = pd.read_sql('SELECT * FROM prism', conn)
prism_check

,county_name,county_fips,year,month,date,precipitation_total_km3,max_temperature_c
0,Wahkiakum,69,2000,1,2000-01-01,0.396535,8.536
1,Wahkiakum,69,2000,2,2000-02-01,0.196691,10.657
2,Wahkiakum,69,2000,3,2000-03-01,0.213708,11.328
3,Wahkiakum,69,2000,4,2000-04-01,0.132705,16.417
4,Wahkiakum,69,2000,5,2000-05-01,0.139677,17.844
...,...,...,...,...,...,...,...
33511,Skamania,59,2020,8,2020-08-01,0.057215,28.443
33512,Skamania,59,2020,9,2020-09-01,0.573716,25.130
33513,Skamania,59,2020,10,2020-10-01,0.993834,18.700
33514,Skamania,59,2020,11,2020-11-01,1.840310,10.701


**Creating table for CDC WONDER mortality data**

In [12]:
#inspecting data frame for SQL table
cdc_hyp = pd.read_csv("cdc-hyperthermia-related-deaths.csv")
cdc_hyp.head()
cdc_hyp.dtypes
#print(prism.columns)

State           object
County          object
County Code    float64
Year           float64
Month          float64
Deaths          object
Population      object
Crude Rate      object
dtype: object

In [13]:
#loading in all the cdc data
cdc_asth = pd.read_csv("cdc-asthma-related-deaths.csv")
cdc_copd = pd.read_csv("cdc-copd-related-deaths.csv")

In [14]:
#renaming dataframe columns to match the SQL tables
cdc_hyp.rename(columns={'County Code': 'county_code', 'Crude Rate': 'crude_rate'}, inplace=True)
cdc_asth.rename(columns={'County Code': 'county_code', 'Crude Rate': 'crude_rate'}, inplace=True)
cdc_copd.rename(columns={'County Code': 'county_code', 'Crude Rate': 'crude_rate'}, inplace=True)

In [15]:
#drop table first if edits are needed
c.execute(""" DROP TABLE IF EXISTS mortality """)

#create new table for mortality data
c.execute(""" CREATE TABLE mortality (
            state VARCHAR(50),
            county INT(10),
            county_code INT(5),
            year FLOAT(5),
            month FLOAT(3),
            deaths VARCHAR(50),
            population VARCHAR(50),
            crude_rate VARCHAR(50)
            ) """)

In [16]:
#filling the table with data
cdc_hyp.to_sql('mortality', conn, if_exists='append', index=False)
cdc_asth.to_sql('mortality', conn, if_exists='append', index=False)
cdc_copd.to_sql('mortality', conn, if_exists='append', index=False)

33560

In [17]:
cdc_check = pd.read_sql('SELECT * FROM mortality', conn)
cdc_check.head()

,state,county,county_code,year,month,deaths,population,crude_rate
0,Washington,"Adams County, WA",53001.0,2000.0,1.0,0,Not Applicable,Not Applicable
1,Washington,"Adams County, WA",53001.0,2000.0,2.0,0,Not Applicable,Not Applicable
2,Washington,"Adams County, WA",53001.0,2000.0,3.0,0,Not Applicable,Not Applicable
3,Washington,"Adams County, WA",53001.0,2000.0,4.0,0,Not Applicable,Not Applicable
4,Washington,"Adams County, WA",53001.0,2000.0,5.0,0,Not Applicable,Not Applicable


In [18]:
conn.commit()

In [19]:
conn.close()